In [1]:
import pandas as pd
import numpy as np

In [2]:
ISO_OUR_COUNTRY_CODES = ['AFG', 'CAF', 'COD', 'COG', 'COL', 'ETH', 'IRQ', 'JOR', 
                         'LBN', 'MMR', 'PAK', 'PSE', 'SOM', 'SSD', 'SDN', 'SYR', 'YEM']

# нужно для датасета conflicts
OUR_COUNTRY_NAMES    = ['Afghanistan', 'Central African Republic', 'DR Congo (Zaire)', 'Congo',
                        'Colombia', 'Ethiopia', 'Iraq', 'Jordan',
                        'Lebanon', 'Myanmar (Burma)', 'Pakistan', "Israel: Palestine",
                        'Somalia', 'South Sudan', 'Sudan', 'Syria', 'Yemen (North Yemen)']
COUNTRY_ISO          = dict(zip(OUR_COUNTRY_NAMES,ISO_OUR_COUNTRY_CODES)) 

# в table_long не хватает {'COG', 'PSE', 'SSD', 'SYR'}
# general_description_2016 = pd.read_csv('data_3sided_model_(coflict_disaster_gdp)/table_long.csv')
gdp_2016                 = pd.read_csv('data_3sided_model_(coflict_disaster_gdp)/gdp_2016.csv')


# CBPF_codes_2016          = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/country_codes.csv')
disasters_2016           = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/disasters_2016.csv')
conflicts                = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/data_3sided_model_(coflict_disaster_gdp)/ged221-csv.zip')
population                = pd.read_csv('./data_3sided_model_(coflict_disaster_gdp)/population.csv')

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## population

In [3]:
population_2016_our = population[['Country Code', 'Country Name',  '2016 [YR2016]']].dropna().rename(columns={'2016 [YR2016]':'population_2016','Country Code':'iso'})
population_2016_our = population_2016_our[population_2016_our['iso'].isin(ISO_OUR_COUNTRY_CODES)].reset_index(drop=True)
population_2016_our

,iso,Country Name,population_2016
0,AFG,Afghanistan,35383028
1,CAF,Central African Republic,4537683
2,COL,Colombia,48175048
3,COD,"Congo, Dem. Rep.",78789130
4,COG,"Congo, Rep.",4980996
...,...,...,...
12,SSD,South Sudan,10832520
13,SDN,Sudan,39847433
14,SYR,Syrian Arab Republic,17465567
15,PSE,West Bank and Gaza,4367088


### gdp

In [4]:
gdp_2016_our = gdp_2016[gdp_2016['Country Code'].isin(ISO_OUR_COUNTRY_CODES)]
gdp_2016_our = gdp_2016_our.rename(columns={'Country Code':'iso','2016':'gdp_2016'})
print(len(ISO_OUR_COUNTRY_CODES))
print(gdp_2016_our.shape)
gdp_2016_our

17
(17, 3)


,Country Name,iso,gdp_2016
2,Afghanistan,AFG,1.811656e+10
34,Central African Republic,CAF,1.825018e+09
43,"Congo, Dem. Rep.",COD,3.713480e+10
44,"Congo, Rep.",COG,1.021934e+10
45,Colombia,COL,2.828250e+11
...,...,...,...
206,Sudan,SDN,1.029437e+11
213,Somalia,SOM,5.529873e+09
216,South Sudan,SSD,0.000000e+00
227,Syrian Arab Republic,SYR,1.245346e+10


### disasters

In [5]:
dis_cols = ['Country', 'ISO', 'Disaster Group', 'Total Affected','Total Deaths',"Total Damages, Adjusted ('000 US$)"]
print(disasters_2016.shape)
disasters_2016.columns

(533, 50)


Index(['Dis No', 'Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'AID Contribution ('000 US$)', 'Dis Mag Value',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')

In [7]:
# нет данных по {'COG', 'LBN', 'PSE', 'SYR'}
df = disasters_2016[dis_cols].fillna(0).groupby(['Country','ISO','Disaster Group']).sum().reset_index()
disasters_2016_our = pd.pivot_table(df,values=["Total Affected","Total Deaths","Total Damages, Adjusted ('000 US$)"], index=['Country', 'ISO'], columns=['Disaster Group'])
disasters_2016_our = disasters_2016_our.reset_index()
disasters_2016_our.columns = ['country','iso','affected_natural','affected_techno','damages_natural','damages_techno','death_natural','death_techno']
disasters_2016_our

,country,iso,affected_natural,affected_techno,damages_natural,damages_techno,death_natural,death_techno
0,Afghanistan,AFG,0.0,20.0,0.0,0.0,84.0,108.0
1,Albania,ALB,4500.0,NaN,0.0,NaN,2.0,NaN
2,Algeria,DZA,NaN,24.0,NaN,0.0,NaN,58.0
3,Angola,AGO,9899.0,NaN,0.0,NaN,113.0,NaN
4,Argentina,ARG,85769.0,NaN,1131264.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...
118,Venezuela (Bolivarian Republic of),VEN,0.0,14.0,0.0,0.0,0.0,11.0
119,Viet Nam,VNM,2503129.0,0.0,955632.0,0.0,189.0,13.0
120,Yemen,YEM,29950.0,0.0,0.0,0.0,51.0,60.0
121,Zambia,ZMB,NaN,0.0,NaN,0.0,NaN,24.0


### conflicts

In [8]:
cols_conf = ['country','country_id','year','deaths_civilians','best']
print('conflicts',conflicts.shape)
display(conflicts.head(1))

conflicts_2016 = conflicts[cols_conf][conflicts.year == 2016]
conflicts_2016_our = conflicts_2016[conflicts_2016.country.isin(OUR_COUNTRY_NAMES)]
conflicts_2016_our = conflicts_2016_our.groupby(['country', 'country_id', 'year']).sum().reset_index()
conflicts_2016_our['iso'] = conflicts_2016_our.country
conflicts_2016_our = conflicts_2016_our.replace({'iso':COUNTRY_ISO})
conflicts_2016_our = conflicts_2016_our.rename(columns={'best':'death_total'})
print('conflicts_2016_our', conflicts_2016_our.shape)
display(conflicts_2016_our)


conflicts (293634, 49)


,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,dyad_dset_id,...,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob
0,244657,IRQ-2017-1-524-322,2017,1,Clear,1,259,259,Iraq: Government,524,...,2017-07-31 00:00:00.000,0,4,0,2,6,6,6,645,NaN


conflicts_2016_our (16, 6)


,country,country_id,year,deaths_civilians,death_total,iso
0,Afghanistan,700,2016,1084,18719,AFG
1,Central African Republic,482,2016,511,688,CAF
2,Colombia,100,2016,3,38,COL
3,Congo,484,2016,42,73,COG
4,DR Congo (Zaire),490,2016,737,2194,COD
...,...,...,...,...,...,...
11,Somalia,520,2016,318,2065,SOM
12,South Sudan,626,2016,354,1303,SSD
13,Sudan,625,2016,790,1994,SDN
14,Syria,652,2016,14290,53863,SYR


### gdp + disasters + conflicts

In [9]:
df_all = gdp_2016_our.merge(disasters_2016_our,left_on='iso',right_on='iso',how='left', suffixes=('', '_y'))
df_all = df_all.drop(df_all.filter(regex='_y$').columns, axis=1)

df_all = df_all.merge(conflicts_2016_our,left_on='iso',right_on='iso',how='left', suffixes=('', '_y'))
df_all = df_all.drop(df_all.filter(regex='_y$').columns, axis=1)

df_all = df_all.merge(population_2016_our,left_on='iso',right_on='iso',how='left', suffixes=('', '_y'))
df_all = df_all.drop(df_all.filter(regex='_y$').columns, axis=1)

df_all = df_all[['year', 'iso','Country Name', 
                'gdp_2016', 'population_2016',
                'affected_natural', 'affected_techno', 'death_natural','death_techno','damages_natural', 'damages_techno',
                'deaths_civilians', 'death_total']]

df_all.year = 2016
df_all = df_all[~(df_all.iso.isin(['PSE','SSD']))].reset_index(drop=True)
df_all

,year,iso,Country Name,gdp_2016,population_2016,affected_natural,affected_techno,death_natural,death_techno,damages_natural,damages_techno,deaths_civilians,death_total
0,2016,AFG,Afghanistan,1.811656e+10,35383028,0.0,20.0,84.0,108.0,0.0,0.0,1084.0,18719.0
1,2016,CAF,Central African Republic,1.825018e+09,4537683,266.0,NaN,21.0,NaN,0.0,NaN,511.0,688.0
2,2016,COD,"Congo, Dem. Rep.",3.713480e+10,78789130,15273.0,22.0,105.0,57.0,0.0,0.0,737.0,2194.0
3,2016,COG,"Congo, Rep.",1.021934e+10,4980996,NaN,NaN,NaN,NaN,NaN,NaN,42.0,73.0
4,2016,COL,Colombia,2.828250e+11,48175048,2284.0,6.0,10.0,140.0,0.0,0.0,3.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2016,PAK,Pakistan,3.136299e+11,203631356,11095.0,526.0,409.0,252.0,2258.0,0.0,384.0,1100.0
11,2016,SDN,Sudan,1.029437e+11,39847433,200636.0,29.0,190.0,16.0,0.0,0.0,790.0,1994.0
12,2016,SOM,Somalia,5.529873e+09,14185635,14165.0,0.0,506.0,106.0,0.0,0.0,318.0,2065.0
13,2016,SYR,Syrian Arab Republic,1.245346e+10,17465567,NaN,NaN,NaN,NaN,NaN,NaN,14290.0,53863.0


In [207]:
df_all.to_csv('data_3sided_model_(coflict_disaster_gdp)/3sided_model_countries.csv',index=False)

### conflicts and disasters analysis

In [10]:
conflicts.columns

Index(['id', 'relid', 'year', 'active_year', 'code_status', 'type_of_violence',
       'conflict_dset_id', 'conflict_new_id', 'conflict_name', 'dyad_dset_id',
       'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id', 'side_a',
       'side_b_dset_id', 'side_b_new_id', 'side_b', 'number_of_sources',
       'source_article', 'source_office', 'source_date', 'source_headline',
       'source_original', 'where_prec', 'where_coordinates',
       'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude',
       'geom_wkt', 'priogrid_gid', 'country', 'country_id', 'region',
       'event_clarity', 'date_prec', 'date_start', 'date_end', 'deaths_a',
       'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best', 'high', 'low',
       'gwnoa', 'gwnob'],
      dtype='object')

In [12]:
conf_analysis = (conflicts[(conflicts.year == 2016) & (conflicts.country.isin(OUR_COUNTRY_NAMES))].groupby(['region','country','dyad_name','latitude', 'longitude'])[['deaths_civilians','best']]
                                                                                  .agg({'best':['count','sum'],'deaths_civilians':['sum']}))
conf_analysis.columns = ['qty_conf','death_total','deaths_civilians']


conf_analysis.to_csv('data_3sided_model_(coflict_disaster_gdp)/analysis_conf.csv',index=False)
conf_analysis

qty_conf  \
region      country                  dyad_name           latitude  longitude             
Africa      Central African Republic 3R - Civilians      6.714140  15.351430         1   
                                                         6.779093  15.293645         1   
                                                         6.845106  15.181429         1   
                                                         6.936024  15.449854         1   
                                                         6.983333  15.650000         1   
...                                                                                ...   
Middle East Yemen (North Yemen)      IS - Forces of Hadi 12.779444 45.036667        16   
                                                         13.056667 44.881944         1   
                                                         13.166667 44.583332         1   
                                                         14.542481 49.124244         3   
                                                         16.000000 49.000000         1   

                                                                              death_total  \
region      country                  dyad_name           latitude  longitude                
Africa      Central African Republic 3R - Civilians      6.714140  15.351430           14   
                                                         6.779093  15.293645            2   
                                                         6.845106  15.181429            2   
                                                         6.936024  15.449854           17   
                                                         6.983333  15.650000           15   
...                                                                                   ...   
Middle East Yemen (North Yemen)      IS - Forces of Hadi 12.779444 45.036667          293   
                                                         13.056667 44.881944            3   
                                                         13.166667 44.583332            3   
                                                         14.542481 49.124244          111   
                                                         16.000000 49.000000            5   

                                                                              deaths_civilians  
region      country                  dyad_name           latitude  longitude                    
Africa      Central African Republic 3R - Civilians      6.714140  15.351430                14  
                                                         6.779093  15.293645                 2  
                                                         6.845106  15.181429                 2  
                                                         6.936024  15.449854                17  
                                                         6.983333  15.650000                15  
...                                                                                        ...  
Middle East Yemen (North Yemen)      IS - Forces of Hadi 12.779444 45.036667                 1  
                                                         13.056667 44.881944                 1  
                                                         13.166667 44.583332                 0  
                                                         14.542481 49.124244                 2  
                                                         16.000000 49.000000                 0  

[3780 rows x 3 columns]

conf_analysis